In [48]:
import pandas as pd
import numpy as np
import os
#import boto3
from datetime import datetime, timezone

#s3_client = boto3.client("s3",
#    aws_access_key_id = os.getenv('ACCESS_KEY'),
#    aws_secret_access_key = os.getenv('SECRET'))
    
#LOCAL_DIR = '/tmp/'

#S3_BUCKET = "testbucketdevynduvall"  # please replace with your bucket name
#CHUNK_SIZE = 10  # determined based on API, memory constraints, experimentation


def get_file_name(export_type):
    dt_now = datetime.now(tz=timezone.utc)
    file_name = (
        export_type
        + "."
        + dt_now.strftime("%Y-%m-%d")

        + ".csv"
    )
    return file_name

# Gets the file from s3
def get_all_rental_data(key):
    #os.chdir(LOCAL_FILE_SYS)
    #s3_client.download_file(S3_BUCKET, str("Rental/" + key), str(LOCAL_FILE_SYS + key))
    #df_rental = pd.read_csv(LOCAL_FILE_SYS + key)
    df_rental = pd.read_csv('Data/Rental.2023-02-02.csv')
    df_rental = df_rental.drop(['has_leadform', 'source', 'page_no', 'rank', 'list_tracking', 'address_new.county_needed_for_uniq','address_new.long', 'has_matterport', 'products', 'photo_count'], axis = 1)
    return df_rental

def get_all_restaurant_rental_data(key):
    # s3_client.download_file(S3_BUCKET, str("Rental_Restaurant/" + key), str(LOCAL_FILE_SYS + key))
    #df_rental_restaurant = pd.read_csv(LOCAL_FILE_SYS + key)
    df_rental_restaurant = pd.read_csv('Data/Rental_Restaurant.2023-02-02.csv')
    return df_rental_restaurant

def join_data(rental, rental_restaurant):
    joined = rental_restaurant.merge(rental, on='property_id', how='left')
    first_column = joined.pop('property_id')
    joined.insert(0, 'property_id', first_column)
    return joined

def upload_to_aws(df, key):
    
    df.to_csv(str(LOCAL_FILE_SYS + key))
    
    s3_client.upload_file(str(LOCAL_FILE_SYS + key), S3_BUCKET, str("Rental_Restaurant_Joined/" + key))


def start():
    key_rental = get_file_name("Rental")
    key_restaurant = get_file_name("Restaurant")
    key_joined = get_file_name("Joined")
    
    df_rental = get_all_rental_data(key_rental)
    df_rental_restaurant = get_all_restaurant_rental_data(key_restaurant)
    
    df_rental_restaurant_all = join_data(df_rental, df_rental_restaurant)
    
    # upload_to_aws(df_rental_restaurant_all, key_joined)

In [51]:
test = start()

In [52]:
test

,property_id,Unnamed: 0_x,business_status,icon,icon_background_color,icon_mask_base_uri,name_x,photos,place_id,rating,...,address_new.county,address_new.lat,address_new.line,address_new.postal_code,address_new.state_code,address_new.state,address_new.time_zone,is_new_listing,sqft_raw,baths_full
0,1096736307,0,OPERATIONAL,https://maps.gstatic.com/mapfiles/place_api/ic...,#7B9EB0,https://maps.gstatic.com/mapfiles/place_api/ic...,West Seattle Bowl,"[{'height': 1365, 'html_attributions': ['<a hr...",ChIJ3Uu9FVBAkFQRcN7l5PP0Hb0,4.6,...,King,47.562724,4502 42nd Ave SW,98116.0,WA,Washington,America/Los_Angeles,NaN,NaN,NaN
1,1096736307,1,OPERATIONAL,https://maps.gstatic.com/mapfiles/place_api/ic...,#4B96F3,https://maps.gstatic.com/mapfiles/place_api/ic...,Easy Street Records & Cafe,"[{'height': 3264, 'html_attributions': ['<a hr...",ChIJEw2sg_lAkFQR25Oh_sq8qRY,4.7,...,King,47.562724,4502 42nd Ave SW,98116.0,WA,Washington,America/Los_Angeles,NaN,NaN,NaN
2,1096736307,2,OPERATIONAL,https://maps.gstatic.com/mapfiles/place_api/ic...,#FF9E67,https://maps.gstatic.com/mapfiles/place_api/ic...,Elliott Bay Brewery & Pub,"[{'height': 3715, 'html_attributions': ['<a hr...",ChIJKdCRYvlAkFQR4o1cPE_IfQI,4.5,...,King,47.562724,4502 42nd Ave SW,98116.0,WA,Washington,America/Los_Angeles,NaN,NaN,NaN
3,1096736307,3,OPERATIONAL,https://maps.gstatic.com/mapfiles/place_api/ic...,#4B96F3,https://maps.gstatic.com/mapfiles/place_api/ic...,The Beer Junction,"[{'height': 1944, 'html_attributions': ['<a hr...",ChIJCV0bcvlAkFQRTZ-xksRsYA8,4.8,...,King,47.562724,4502 42nd Ave SW,98116.0,WA,Washington,America/Los_Angeles,NaN,NaN,NaN
4,1096736307,4,OPERATIONAL,https://maps.gstatic.com/mapfiles/place_api/ic...,#FF9E67,https://maps.gstatic.com/mapfiles/place_api/ic...,Talarico's Pizzeria West Seattle,"[{'height': 1200, 'html_attributions': ['<a hr...",ChIJPdI3Y_lAkFQRM85l0DegluY,4.5,...,King,47.562724,4502 42nd Ave SW,98116.0,WA,Washington,America/Los_Angeles,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
121,9595985456,121,OPERATIONAL,https://maps.gstatic.com/mapfiles/place_api/ic...,#FF9E67,https://maps.gstatic.com/mapfiles/place_api/ic...,Aceituno's Mexican Food,"[{'height': 4160, 'html_attributions': ['<a hr...",ChIJ88NRBXoRkFQR4-u8OKeBKQM,3.9,...,King,47.722409,3025 NE 130th St,98125.0,WA,Washington,America/Los_Angeles,True,NaN,NaN
122,9595985456,122,OPERATIONAL,https://maps.gstatic.com/mapfiles/place_api/ic...,#FF9E67,https://maps.gstatic.com/mapfiles/place_api/ic...,Beyond The Bowl,"[{'height': 4032, 'html_attributions': ['<a hr...",ChIJ82FrTXsRkFQRV0-zmKJn4CE,4.6,...,King,47.722409,3025 NE 130th St,98125.0,WA,Washington,America/Los_Angeles,True,NaN,NaN
123,9595985456,123,OPERATIONAL,https://maps.gstatic.com/mapfiles/place_api/ic...,#FF9E67,https://maps.gstatic.com/mapfiles/place_api/ic...,2CThai Bistro & Spirits,"[{'height': 3024, 'html_attributions': ['<a hr...",ChIJL9YIrWQRkFQR0zuF8rl6bV8,4.6,...,King,47.722409,3025 NE 130th St,98125.0,WA,Washington,America/Los_Angeles,True,NaN,NaN
124,9595985456,124,OPERATIONAL,https://maps.gstatic.com/mapfiles/place_api/ic...,#FF9E67,https://maps.gstatic.com/mapfiles/place_api/ic...,SPS school lunch delivery 11:22am M-F,NaN,ChIJnX6q3ucRkFQRr7xqWECrHx0,NaN,...,King,47.722409,3025 NE 130th St,98125.0,WA,Washington,America/Los_Angeles,True,NaN,NaN
